# Financial Report Analysis - Interactive Example

This notebook demonstrates the complete workflow of the Financial Report Analyzer:
1. PDF Parsing
2. GAAP/IFRS Classification
3. Financial Model Building
4. Forecasting
5. Report Generation

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.core.pdf_parser import PDFParser
from src.core.gaap_ifrs_classifier import GaapIfrsClassifier
from src.core.model_engine import ModelEngine
from src.core.forecast_engine import ForecastEngine
from src.core.report_generator import ReportGenerator
from src.models.schemas import *

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## Step 1: Parse PDF Financial Report

In [ ]:
# Parse PDF (replace with your PDF path)
pdf_path = "../data/sample_reports/sample_10k.pdf"

# Uncomment to use real PDF:
# parser = PDFParser(pdf_path)
# extracted_data = parser.extract()

# For demo, we'll use sample data
print("Using sample data for demonstration")
print("To use real PDF, uncomment the parser code above")

## Step 2: Classify Accounting Standard

In [ ]:
# Sample text for classification
sample_text = """
The consolidated financial statements have been prepared in accordance with 
generally accepted accounting principles in the United States (US GAAP).
We use the LIFO method for inventory valuation.
"""

classifier = GaapIfrsClassifier()
standard, confidence, evidence = classifier.classify(sample_text)

print(f"Detected Standard: {standard.value}")
print(f"Confidence: {confidence:.1%}")
print(f"\nEvidence:")
print(f"  GAAP Score: {evidence['gaap_score']}")
print(f"  IFRS Score: {evidence['ifrs_score']}")
print(f"\nGAAP Evidence: {evidence['gaap_evidence'][:3]}")

## Step 3: Create Sample Financial Statements

In [ ]:
# Create sample statements
statements = FinancialStatements(
    company_name="TechCorp Inc.",
    ticker="TECH",
    fiscal_year=2023,
    report_type=ReportType.FORM_10K,
    accounting_standard=AccountingStandard.GAAP,
    currency=Currency.USD
)

# Generate 3 years of historical data
for year in range(2021, 2024):
    base_revenue = 1_000_000_000 * (1.1 ** (year - 2021))
    
    # Income Statement
    income = IncomeStatement(
        period_start=date(year, 1, 1),
        period_end=date(year, 12, 31),
        revenue=base_revenue,
        cost_of_revenue=base_revenue * 0.60,
        gross_profit=base_revenue * 0.40,
        operating_expenses=base_revenue * 0.20,
        operating_income=base_revenue * 0.20,
        depreciation_amortization=base_revenue * 0.05,
        net_income=base_revenue * 0.15
    )
    statements.income_statements.append(income)
    
    # Balance Sheet
    bs = BalanceSheet(
        period_end=date(year, 12, 31),
        cash_and_equivalents=base_revenue * 0.15,
        accounts_receivable=base_revenue * 0.12,
        inventory=base_revenue * 0.10,
        total_current_assets=base_revenue * 0.40,
        property_plant_equipment_net=base_revenue * 0.50,
        total_assets=base_revenue * 1.20,
        accounts_payable=base_revenue * 0.08,
        total_current_liabilities=base_revenue * 0.15,
        total_liabilities=base_revenue * 0.45,
        retained_earnings=base_revenue * 0.60,
        total_shareholders_equity=base_revenue * 0.75
    )
    statements.balance_sheets.append(bs)
    
    # Cash Flow Statement
    cf = CashFlowStatement(
        period_start=date(year, 1, 1),
        period_end=date(year, 12, 31),
        net_income=income.net_income,
        depreciation_amortization=income.depreciation_amortization,
        cash_from_operations=income.net_income + income.depreciation_amortization,
        capital_expenditures=-base_revenue * 0.06,
        cash_from_investing=-base_revenue * 0.06
    )
    statements.cash_flow_statements.append(cf)

print(f"✓ Created {len(statements.income_statements)} years of financial data")

## Step 4: Build Linked 3-Statement Model

In [ ]:
# Build linked model
model_engine = ModelEngine(statements)
linked_model = model_engine.build_linked_model()

print(f"Model Balanced: {linked_model.is_balanced}")
print(f"Validation Errors: {len(linked_model.validation_errors)}")

if linked_model.validation_errors:
    print("\nErrors:")
    for error in linked_model.validation_errors[:5]:
        print(f"  - {error}")

# Get summary metrics
summary = model_engine.get_summary_metrics()
print("\nSummary Metrics:")
for key, value in summary.items():
    if isinstance(value, (int, float)) and value > 1000:
        print(f"  {key}: ${value:,.0f}")
    elif isinstance(value, float):
        print(f"  {key}: {value:.1%}")
    else:
        print(f"  {key}: {value}")

## Step 5: Visualize Historical Performance

In [ ]:
# Create DataFrame for visualization
years = [stmt.period_end.year for stmt in statements.income_statements]
revenues = [stmt.revenue / 1e9 for stmt in statements.income_statements]
net_incomes = [stmt.net_income / 1e9 for stmt in statements.income_statements]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Revenue trend
ax1.plot(years, revenues, marker='o', linewidth=2, markersize=8, color='#1f77b4')
ax1.set_title('Revenue Trend', fontsize=14, fontweight='bold')
ax1.set_xlabel('Year')
ax1.set_ylabel('Revenue ($B)')
ax1.grid(True, alpha=0.3)

# Net Income trend
ax2.plot(years, net_incomes, marker='s', linewidth=2, markersize=8, color='#2ca02c')
ax2.set_title('Net Income Trend', fontsize=14, fontweight='bold')
ax2.set_xlabel('Year')
ax2.set_ylabel('Net Income ($B)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 6: Generate Forecast

In [ ]:
# Define forecast assumptions
assumptions = ForecastAssumptions(
    revenue_growth_rate=0.08,  # 8% annual growth
    gross_margin=0.42,
    operating_margin=0.22,
    tax_rate=0.21,
    capex_percent_of_revenue=0.06,
    days_sales_outstanding=45,
    days_inventory_outstanding=60,
    days_payable_outstanding=35
)

# Generate forecast
forecast_engine = ForecastEngine(linked_model)
forecast_model = forecast_engine.forecast(
    years=5,
    assumptions=assumptions,
    scenario="base"
)

print(f"✓ Generated {forecast_model.forecast_years}-year forecast")
print(f"\nForecast Revenue (Year 5): ${forecast_model.forecast_income_statements[-1].revenue:,.0f}")
print(f"Forecast Net Income (Year 5): ${forecast_model.forecast_income_statements[-1].net_income:,.0f}")

## Step 7: Visualize Historical + Forecast

In [ ]:
# Combine historical and forecast data
all_income = forecast_model.historical_income_statements + forecast_model.forecast_income_statements

years = [stmt.period_end.year for stmt in all_income]
revenues = [stmt.revenue / 1e9 for stmt in all_income]
net_incomes = [stmt.net_income / 1e9 for stmt in all_income]

# Split into historical and forecast
hist_years = years[:len(forecast_model.historical_income_statements)]
hist_revenues = revenues[:len(forecast_model.historical_income_statements)]
forecast_years = years[len(forecast_model.historical_income_statements)-1:]
forecast_revenues = revenues[len(forecast_model.historical_income_statements)-1:]

plt.figure(figsize=(12, 6))
plt.plot(hist_years, hist_revenues, marker='o', linewidth=2, 
         markersize=8, label='Historical', color='#1f77b4')
plt.plot(forecast_years, forecast_revenues, marker='o', linewidth=2, 
         markersize=8, linestyle='--', label='Forecast', color='#ff7f0e')
plt.axvline(x=hist_years[-1], color='gray', linestyle=':', alpha=0.5)
plt.title('Revenue: Historical vs Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Revenue ($B)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Step 8: Analyze Financial Ratios

In [ ]:
# Create ratios DataFrame
all_ratios = forecast_model.historical_ratios + forecast_model.forecast_ratios

ratios_data = {
    'Year': [r.period.year for r in all_ratios],
    'Gross Margin': [r.gross_margin for r in all_ratios],
    'Operating Margin': [r.operating_margin for r in all_ratios],
    'Net Margin': [r.net_margin for r in all_ratios],
    'ROE': [r.return_on_equity for r in all_ratios],
    'Current Ratio': [r.current_ratio for r in all_ratios]
}

df_ratios = pd.DataFrame(ratios_data)
print("Financial Ratios:")
print(df_ratios.to_string(index=False))

## Step 9: Generate PDF Report

In [ ]:
# Generate comprehensive PDF report
report_generator = ReportGenerator(forecast_model)
output_path = "../output/financial_analysis_report.pdf"

report_generator.generate_pdf(output_path)

print(f"✓ PDF Report generated: {output_path}")
print("\nReport includes:")
print("  - Executive Summary")
print("  - Historical Financial Statements")
print("  - Linked 3-Statement Model")
print("  - 5-Year Forecast")
print("  - Financial Ratios & KPIs")
print("  - Charts & Visualizations")

## Step 10: Scenario Analysis

In [ ]:
# Generate multiple scenarios
scenarios = {}

for scenario in ['base', 'bull', 'bear']:
    engine = ForecastEngine(linked_model)
    model = engine.forecast(years=5, scenario=scenario)
    scenarios[scenario] = model

# Plot scenario comparison
plt.figure(figsize=(12, 6))

for scenario_name, model in scenarios.items():
    all_income = model.historical_income_statements + model.forecast_income_statements
    years = [stmt.period_end.year for stmt in all_income]
    revenues = [stmt.revenue / 1e9 for stmt in all_income]
    
    # Only plot forecast portion
    forecast_start = len(model.historical_income_statements) - 1
    plt.plot(years[forecast_start:], revenues[forecast_start:], 
             marker='o', linewidth=2, label=scenario_name.capitalize())

plt.title('Scenario Analysis: Revenue Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Revenue ($B)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nScenario Comparison (Year 5 Revenue):")
for scenario_name, model in scenarios.items():
    final_revenue = model.forecast_income_statements[-1].revenue
    print(f"  {scenario_name.capitalize()}: ${final_revenue:,.0f}")